In [1]:
import pandas as pd
import numpy as np

In [36]:
train_df = pd.read_csv("/home/sharedrive/nafi/trafficp2/datasets/itu_train.csv")
train_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,68855579,10,6,1038,11595,346,0,103.800000,167.133879,...,32,998.0,0.0,998,998,68300000.0,0.0,68300000,68300000,DoS Hulk
1,53,196,2,2,70,174,35,35,35.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,123,118,2,2,96,96,48,48,48.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,80,295657,7,10,1114,15841,1084,0,159.142857,407.829796,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,80,15705351,7,5,407,452,377,0,58.142857,140.620563,...,20,360718.0,0.0,360718,360718,9767208.0,0.0,9767208,9767208,BENIGN


In [37]:
test_df = pd.read_csv("/home/sharedrive/nafi/trafficp2/datasets/itu_test.csv")
test_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,998,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS Hulk
1,80,63111103,7,0,0,0,0,0,0.0,0.0,...,40,7015565.0,0.0,7015565,7015565,18700000.0,12200000.0,32100000,8015910,DoS Slowhttptest
2,53,202,2,2,98,130,49,49,49.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,80,3,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,DoS Hulk
4,7200,37,1,1,0,6,0,0,0.0,0.0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,PortScan


In [38]:
train_df.shape, test_df.shape

((1783356, 79), (512077, 79))

In [39]:
def change(df):
    column_names = list(df.columns)
    new_column_names = [name.strip().replace(" ", "_") for name in column_names]
    _dict = {k:v for k,v in zip(column_names, new_column_names)}
    df.rename(columns = _dict, inplace = True)
    labels = df["Label"].unique().tolist()
    new_labels = [''.join(filter(lambda x: ord(x)<128, l.strip().replace(" ","_"))) for l in labels]
    _dict = {k:v for k,v in zip(labels, new_labels)}
    df["Label"].replace(to_replace=_dict, inplace=True)
    return df

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

In [40]:
train_df = change(train_df)
test_df = change(test_df)

In [41]:
selected_features = ['Flow_Duration', 'Total_Length_of_Bwd_Packets', 'Bwd_Packet_Length_Max',
                       'Bwd_Packet_Length_Mean', 'Flow_Bytes/s', 'Flow_Packets/s',
                       'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min',
                       'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max',
                       'Fwd_IAT_Min', 'Fwd_Packets/s', 'Bwd_Packets/s', 'Max_Packet_Length',
                       'Packet_Length_Mean', 'Average_Packet_Size', 'Avg_Bwd_Segment_Size',
                       'Subflow_Bwd_Bytes', 'Init_Win_bytes_forward', 'min_seg_size_forward']

In [42]:
def feature_selection(df, save_as):
    reduced_df = df[selected_features + ["Label"]]
    x = clean_dataset(reduced_df.drop(columns=["Label"]))
    y = reduced_df["Label"].loc[x.index]
    reduced_df = pd.concat([x, y], axis=1)
    reduced_df.to_csv(f"./datasets/feature_selected_{save_as}.csv", index=False)

In [43]:
feature_selection(train_df, "itu_train")

In [44]:
feature_selection(test_df, "itu_test")

After removing negative values....

In [4]:
itu_train = pd.read_csv("/home/sharedrive/nafi/trafficp3/datasets/feature_selected_itu_train.csv")
itu_test = pd.read_csv("/home/sharedrive/nafi/trafficp3/datasets/feature_selected_itu_test.csv")
itu_train.shape, itu_test.shape

((1781545, 25), (511540, 25))

In [5]:
itu_train.drop(columns=["Init_Win_bytes_forward", "Flow_IAT_Min"], inplace=True)
itu_test.drop(columns=["Init_Win_bytes_forward", "Flow_IAT_Min"], inplace=True)
itu_train.shape, itu_test.shape

((1781545, 23), (511540, 23))

In [6]:
def replace_negatives_with_positive_mean(df):
    df_copy = df.copy()
    
    for col in df_copy.columns:
        if col == "Label":
            continue
        if pd.api.types.is_numeric_dtype(df_copy[col]):
            # calculate the mean of positive values in the column
            positive_values = df_copy[col][df_copy[col] >= 0]
            if not positive_values.empty:
                positive_mean = positive_values.mean()
                # replace negatives with the mean of positive values
                df_copy.loc[df_copy[col] < 0, col] = positive_mean
            else:
                print(f"Column '{col}' has no positive values. Negatives remain unchanged.")
    
    return df_copy

itu_train = replace_negatives_with_positive_mean(itu_train)

import pandas as pd
from tqdm import tqdm

def analyze_negative_values(df, show_values=False, sample=10):
    # Select only numeric columns to optimize processing
    numeric_cols = df.select_dtypes(include=['number']).columns
    summary = []
    
    # Use tqdm for progress bar (only in Jupyter)
    for col in tqdm(numeric_cols, desc="Processing columns"):
        negative_count = (df[col] < 0).sum()
        result = {"Feature": col, "Negative Count": negative_count}
        
        if show_values and negative_count > 0:
            negatives = df[col][df[col] < 0]
            result["Sample Values"] = negatives.head(sample).tolist()
        
        summary.append(result)
    
    return pd.DataFrame(summary)

analyze_negative_values(itu_train)

Processing columns: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 480.74it/s]


,Feature,Negative Count
0,Flow_Duration,0
1,Total_Length_of_Bwd_Packets,0
2,Bwd_Packet_Length_Max,0
3,Bwd_Packet_Length_Mean,0
4,Flow_Bytes/s,0
5,Flow_Packets/s,0
6,Flow_IAT_Mean,0
7,Flow_IAT_Std,0
8,Flow_IAT_Max,0
9,Fwd_IAT_Total,0


In [7]:
itu_test = replace_negatives_with_positive_mean(itu_test)
analyze_negative_values(itu_test)

Processing columns: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 1296.14it/s]


,Feature,Negative Count
0,Flow_Duration,0
1,Total_Length_of_Bwd_Packets,0
2,Bwd_Packet_Length_Max,0
3,Bwd_Packet_Length_Mean,0
4,Flow_Bytes/s,0
5,Flow_Packets/s,0
6,Flow_IAT_Mean,0
7,Flow_IAT_Std,0
8,Flow_IAT_Max,0
9,Fwd_IAT_Total,0


In [9]:
itu_train.to_csv(f"/home/sharedrive/nafi/trafficp3/datasets/feature_selected_neg_itu_train.csv", index=False)

In [10]:
itu_test.to_csv(f"/home/sharedrive/nafi/trafficp3/datasets/feature_selected_neg_itu_test.csv", index=False)
